# Vulnerable AWS app

Let's hunt for vulnerabilities in an app that use AWS SDK. We begin by setting up the basics for joern lib.

In [1]:
import asyncio

from joern_lib import client, workspace
from joern_lib.detectors import common as cpg, js
from joern_lib.utils import print_table

joern_host = "http://joern:9000"
cpggen_host = "http://cpggen:7072"
joern_username = "admin"
joern_password  = "admin"

## Clone the vulnerable app

We use the vulnerable app from [HooliCorp](https://github.com/HooliCorp/vulnerable-aws-koa-app) for this notebook. We can clone this repo using GitPython and import to Joern using the import_code method.

In [3]:
import git
import os

if not os.path.exists("/tmp/aws-koa"):
    try:
        git.Repo.clone_from("https://github.com/HooliCorp/vulnerable-aws-koa-app.git", "/tmp/aws-koa", branch="main", depth=1)
    except Exception as e:
        print(e)
else:
    print("/tmp/aws-koa already exists")

/tmp/aws-koa already exists


In [4]:
async def create_workspace():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await workspace.import_code(connection, "/tmp/aws-koa", "aws-koa");
asyncio.run(create_workspace())

╭─────────── CPGQL Query ────────────╮
│ os.exists(os.Path("/tmp/aws-koa")) │
╰────────────────────────────────────╯
╭───────────── CPGQL Query ─────────────╮
│ importCode("/tmp/aws-koa", "aws-koa") │
╰───────────────────────────────────────╯
False


## Identify AWS services used

To look for AWS related vulnerabilities, we need to identify the services used by this application. JavaScript detector in joern-lib includes a method called `list_aws_modules` for this purpose.

In [6]:
async def detect_aws_services():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await js.list_aws_modules(connection)
    print_table(res, title="Imported AWS modules")
    res = await js.used_aws_modules(connection)
    print_table(res, title="Used AWS modules")

asyncio.run(detect_aws_services())

╭───────────────────────────────────────────────────────────────────────────────── CPGQL Query ──────────────────────────────────────────────────────────────────────────────────╮
│ cpg.call.assignment.code(".*require\\(.*").argument.isIdentifier.filter(_.typeFullName.contains("aws")).map(t => (t, t.location)).filter(_._1.isIdentifier).dedup.toJsonPretty │
╰────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                                Imported AWS modules                                                                                
┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ dynami ┃        ┃        ┃        ┃       ┃        ┃        ┃        ┃      ┃        ┃        ┃    

## Insecure Authentication

We can see the use of few services such as EC2, S3 but how is the application authenticating with these services? As per [AWS](https://docs.aws.amazon.com/sdk-for-javascript/v2/developer-guide/setting-credentials.html), applications should not hardcode the credentials so let's begin our search here. In the below snippet, we look for the pattern `new Credentials()`.

In [8]:
async def safe_credentials_use():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.p(connection, """cpg.call.name("<operator>.new").code(".*Credentials.*").argument.order(3)""", title="AWS Credentials")
asyncio.run(safe_credentials_use())

╭───────────────────────────────────── CPGQL Query ──────────────────────────────────────╮
│ cpg.call.name("<operator>.new").code(".*Credentials.*").argument.order(3).toJsonPretty │
╰────────────────────────────────────────────────────────────────────────────────────────╯
                                                                                  AWS Credentials                                                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ dynamicTypeHintFullName          ┃ code                                                 ┃ typeFullName      ┃    order ┃ _label    ┃      argumentIndex ┃     lineNumber ┃    id ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│  

From the table, we can see several hardcoded credentials in use in this application. The snippet conveniently shows the use of `p` or `print` method which accepts native CPGQL query and displays the query result as a table.

## Attacker controlled input

Next up is to determine whether any external attacker controlled input is used directly to interact with AWS services without validation or sanitization. For this we need to know some sources (or entrypoints) and sinks (or exitpoints) in the application. We can use the library method `get_http_sources` for this purpose.

In [11]:
async def find_sources():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    sources = await js.get_http_sources(connection)
    print (len(sources))
asyncio.run(find_sources())

╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────────────────────╮
│ cpg.call.code("(?i)(?s)(?i).*(req|ctx)\\.(originalUrl|path|protocol|route|secure|signedCookies|stale|subdomains|xhr|app|pipe|file|files|baseUrl|fresh|hostname|ip|url|ips|method │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
63


For the sinks, we need a generic pattern for AWS sdk. The pattern `.*(send|upload|create|get|delete|list).*` should suffice for our use case.

In [13]:
async def find_reachable_flows():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.call.assignment.code(".*(context|ctx)\\.request.*").argument.isIdentifier', 'cpg.call.code(".*(send|get|create|upload|delete|execute).*").argument.isIdentifier')
asyncio.run(find_reachable_flows())

╭─────────────────────────────────────── CPGQL Query ────────────────────────────────────────╮
│ def source = cpg.call.assignment.code(".*(context|ctx)\\.request.*").argument.isIdentifier │
╰────────────────────────────────────────────────────────────────────────────────────────────╯
╭───────────────────────────────────────── CPGQL Query ─────────────────────────────────────────╮
│ def sink = cpg.call.code(".*(send|get|create|upload|delete|execute).*").argument.isIdentifier │
╰───────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_ca1th4mg.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ vmSpecification       │
│ input        

## Closing Thoughts

This notebook demonstrated the mixed use of built-in library methods and native query methods to interactively identify vulnerabilities in this AWS application.